In [0]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os

In [0]:
filepath = 'drive/My Drive/fyp/bert_abstract_all.7z'


In [0]:
max_seq_len = 10
embedd_len = 768

In [0]:
!pip install pyunpack
!pip install patool



In [0]:
from pyunpack import Archive
Archive(filepath).extractall('')


In [0]:
abstract_path = 'bert_abstract_all.json'

In [0]:
import json
with open(abstract_path) as json_file: 
  abstracts = json.load(json_file)
  print(len(abstracts))

28284


In [0]:
print(len(abstracts[0]['abstract_sentence_all']))

3


In [0]:
!pip install --upgrade tensorflow==1.13.1
!pip install --upgrade pandas
#!pip install -q tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf
import pandas as pd

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [0]:
print(range(4))

range(0, 4)


In [0]:
print(type(abstracts[0]))

<class 'dict'>


In [0]:
def pad_paper(paper, max_seq_length, vec_length):
  padding = vec_length * [0]
  paper_len = len(paper)
  outpaper = paper
  
  if (paper_len > max_seq_length):
    outpaper = paper[:max_seq_length]
    
  elif(paper_len < max_seq_length):
    pad_no = max_seq_length - paper_len
    
    for pad in range(pad_no):
      outpaper.append(padding) 
    
  else:
    outpaper = paper
  return outpaper

In [0]:
new_abs = []
count = 0
size = len(abstracts)
for ab in abstracts:
    vectors = ab['abstract_sentence_all']
    new_vectors = pad_paper(vectors, 10, 768)
    new_ab = ab 
    new_ab['abstract_sentence_all'] = new_vectors
    new_abs.append(new_ab)
    count += 1
    if(count % 1000 == 0):
      print(count, '/', size )

In [0]:
abstracts = new_abs

In [0]:
for ab in abstracts[0]['abstract_sentence_all']:
  print(ab)

In [0]:
##find sequence length
##df['abstract_sentence_all'].str.len().mean()

In [0]:
df = pd.DataFrame(abstracts)

In [0]:
df.head()

,abstract_sentence_all,label,paperId
0,"[[0.4155263900756836, 0.05221998691558838, -0....",evolutionary biology,262147
1,"[[0.9820002913475037, 0.08879165351390839, -0....",molecular biology,393223
2,"[[0.9408681392669678, 0.14392858743667603, -0....",molecular biology,262154
3,"[[0.7481150031089783, 0.2712785005569458, -0.0...",genetics,393231
4,"[[0.17487555742263794, 0.20638687908649445, 0....",bioinformatics,262162


In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
#split training and testing data (80:20)
from sklearn.model_selection import train_test_split
trainDf, testDf = train_test_split(df,stratify=df['label'], test_size = 0.3, random_state = 11)

trainDf.shape,testDf.shape

((19798, 3), (8486, 3))

In [0]:
X_train = trainDf['abstract_sentence_all']
y_train = trainDf['label']
X_test = testDf['abstract_sentence_all'] 
y_test = testDf['label']

In [0]:
df[df['abstract_sentence_all']==df['abstract_sentence_all'].max()]

In [0]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=X_train,
    y=y_train,
    batch_size=64,
    num_epochs=1,
    shuffle=True,
    queue_capacity=10000,
    num_threads=1,
    target_column='label'
)

test_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=X_test,
    y=y_test,
    batch_size=64,
    num_epochs=1,
    shuffle=True,
    queue_capacity=10000,
    num_threads=1,
    target_column='label'
)

In [0]:
df.shape

(28284, 3)

In [0]:
df['label'].values

array(['evolutionary biology', 'molecular biology', 'molecular biology',
       ..., 'bioinformatics', 'microbiology', 'neuroscience'],
      dtype=object)

In [0]:
(X_train.shape,  y_train.shape)

((19798,), (19798,))

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [0]:
test01 = ('i am a boy','label1')
test02 = ('i am a boy','label1')
test03 = ('i am a girl','label2')
test04 = ('i am a girl','label2')
test_data = [test01,test02,test03,test04]

columns = ['feature','label']
t_df = pd.DataFrame(data = test_data, columns = columns)

In [0]:

test01 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],'label1')
test02 = ([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[2,2,2,2]],'label2')
test03 = ([[1,1,1,1],[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]],'label2')
test04 = ([[1,1,1,1],[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]],'label2')
test_data = [test01,test02,test03,test04]

columns = ['feature','label']
t_df = pd.DataFrame(data = test_data, columns = columns)

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
X = t_df.feature
Y = t_df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
X_sequences = X_train.to_numpy()

In [0]:
max_words = 6
max_len = 5
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
sequences_matrix.shape

(3, 5)

In [0]:
import numpy as np

In [0]:
blep = t_df['feature'].to_numpy()
[len(i) for i in blep]

[5, 5, 5, 5]

In [0]:
blep

array([list([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]),
       list([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 0, 0]]),
       list([[1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]),
       list([[1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])],
      dtype=object)

In [0]:
a[0]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [0]:
X_sequences.shape

(3,)

In [0]:
X_train = t_df['feature'][:2].values
y_train = t_df['label'][:2].values

X_test = t_df['feature'][2:].values
y_test = t_df['label'][2:].values

print(X_train)
print(X_test)


[list([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
 list([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 0, 0]])]
[list([[1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
 list([[1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])]


In [0]:
import numpy as np
X = np.array([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
X_train = X.reshape(1, 3, 3) # X.reshape(samples, timesteps, features)

X_train[0]

array([[10, 20, 30],
       [40, 50, 60],
       [70, 80, 90]])

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(([i for i in t_df['feature']], t_df['label']))
    
    
    

In [0]:
test_dataset = dataset.take(2) 
train_dataset = dataset.skip(2)

In [0]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64,input_shape=(5,4)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
type(classes)

set

In [0]:
x1, y1 = generator(X_train, y_train, 64)

TypeError: ignored

In [0]:
history = model.fit_generator(generator(X_train, y_train, BATCH_SIZE),
                    steps_per_epoch=int(len(X_train)/BATCH_SIZE) + 1,
                    epochs=10,
                    validation_data=generator(X_test, y_test, BATCH_SIZE),
                    validation_steps=int(len(X_test)/BATCH_SIZE) + 1)

NameError: ignored

In [0]:
dataset.output_types

In [0]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()

In [0]:
value = sess.run(next_element)

In [0]:
value

(array([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]], dtype=int32), b'label1')

In [0]:
t_df['feature'].values[0]

[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]

In [0]:
type(t_df['feature'].values[0])

TypeError: ignored

In [0]:
data_tf1 = tf.placeholder(tf.float32, [None, 39,1]) 

In [0]:
data_tf = tf.convert_to_tensor(t_df['feature'].values)

TypeError: ignored

In [0]:
#t_df.head()
#Features,labels = t_df,t_df.pop('label')

(Features.shape,labels.shape)

((4, 1), (4,))

In [0]:
type(labels)

pandas.core.series.Series

In [0]:
import numpy as np

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((t_df['feature'], t_df['label']))

TypeError: ignored

In [0]:
##test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))

KeyboardInterrupt: ignored

In [0]:
from numpy import array
test01 = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
test02 = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[2,2,2,2]]
test03 = [[1,1,1,1],[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]]
test04 = [[1,1,1,1],[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]]
test_data = [test01,test02,test03,test04]
labels = [1,1,2,2]
columns = ['feature','label']
t_df = pd.DataFrame(data = (test_data,labels), columns = columns)

NameError: ignored

In [0]:
arr = [[[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],[[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]],[[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]]

In [0]:
arr

In [0]:
W = np.array(arr)
W.shape

(3, 5, 4)